# 401k Contribution

1. In this example I used Integer Programming, AMPL and Python to optimally allocate contributions to Roth and Traditional.
2. Input: 
    * Roth Annual Max = 5500 (default)
    * Total Annual Contribution = Traditional + Roth = 18500 (default)
3. Ouput:
    * Bi-Weekly dollar amount to contribute to Roth
    * Bi-Weekly dollar amount to contribute to Traditional
    * Surplus 
4. Objective Function: Maximize contribution to Roth, remianing contribute to Traditional. Any amount remaining after contributing Roth and Traditional, assign it to Surplus.  
5. Surplus amount would be something we need to contribute lumpsum in a single paycheck. Surplus is the amount of dollars we could not allocate to Roth and Traditional in \\$1 increments. If we were to split the Surplus equally in each paycheck that would be a fractional dollar (cents). My investment plan does not allow for contributing in cents. The minimum contribution has to be in \\$1 increments. 

In [27]:
NUMBER_OF_MONTHS = 11
ROTH_CONTRIBUTION_SO_FAR = 229*2
#TRADITIONAL_CONTRIBUTION_SO_FAR = 
ROTH_MAX = 5500
MAX_CONTRIBUTION = 19000 #Max 401k contribution

In [28]:
model = """
#PART 1: DECISION VARIABLES
var roth >= 0 integer;
var traditional >= 0 integer;
var surplus >= 0 integer;

#PART 2: OBJECTIVE FUNCTION
maximize z: roth*2*{NUMBER_OF_MONTHS}*2 + traditional*2*{NUMBER_OF_MONTHS} + surplus;
#The second 2 here is because I value roth twice more than traditional. 

#PART 3: CONSTRAINTS
s.t. M1: roth*2*{NUMBER_OF_MONTHS}  <= {ROTH_MAX};
s.t. M2: roth*2*{NUMBER_OF_MONTHS} + traditional*2*{NUMBER_OF_MONTHS} + surplus = {MAX_CONTRIBUTION};
""".format(ROTH_MAX=ROTH_MAX, MAX_CONTRIBUTION=MAX_CONTRIBUTION, NUMBER_OF_MONTHS=NUMBER_OF_MONTHS)

In [29]:
from amplpy import AMPL, Environment

ampl = AMPL(Environment('/opt/ampl.linux64'))
print(ampl.getOption('solver'))
ampl.setOption('solver','cplex')
print(ampl.getOption('solver'))
#ampl.read('example.mod') #read the model
ampl.eval(model)
ampl.eval('objective z; solve;')
#ampl.display('z', 'roth', 'traditional', 'surplus')

minos
cplex
CPLEX 12.8.0.0: optimal integer solution; objective 24500
0 MIP simplex iterations
0 branch-and-bound nodes


In [30]:
z=ampl.getValue('z')
roth=ampl.getValue('roth')
traditional=ampl.getValue('traditional')
surplus=ampl.getValue('surplus')

annual_roth_amt = roth*NUMBER_OF_MONTHS*2
annual_traditional_amt = traditional*NUMBER_OF_MONTHS*2

In [31]:
pstring = """
objective function = {z}
bi-weekly roth contribution = ${roth}
bi-weekly traditional contribution = ${traditional}
surplus (amount left after optimal allocation) = ${surplus}
total (without surplus)  = ${total}
total (with surplus)  = ${totalS}
"""


print(pstring.format(z=z,
                     roth=roth,
                     traditional=traditional,
                     surplus=surplus,
                     total=annual_roth_amt+annual_traditional_amt,
                     totalS=annual_roth_amt+annual_traditional_amt+surplus))


objective function = 24500.0
bi-weekly roth contribution = $250.0
bi-weekly traditional contribution = $613.0
surplus (amount left after optimal allocation) = $14.0
total (without surplus)  = $18986.0
total (with surplus)  = $19000.0

